In [5]:
# caga de archivos CSV sobre Azure storage
from ftplib import FTP
from hdbcli import dbapi
import pyodbc
import os.path
import os
import pandas as pd
import logging
from azure.storage.blob import BlobClient, BlobServiceClient, ContainerClient
#from azure.storage.blob.aio import BlobClient

connect_str = os.environ["AZURE_STORAGE_CONNECTION_STRING"]

# conexiones a Azure
def init_SQL():
    server = os.environ["sql_host"]
    database = os.environ["sql_db"] 
    username = os.environ["sql_user"]
    password = os.environ["sql_pwd"]
    connSQL = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';AUTOCOMMIT=FALSE;UID='+username+';PWD='+ password)
    return connSQL
    
def init_FTP():
    Host = os.environ["ftp_host"]
    Puerto = os.environ["ftp_port"]
    Cuenta = os.environ["ftp_cuenta"]
    Pass = os.environ["ftp_pwd"]
    connFTP = [Host, Puerto, Cuenta, Pass]
    return connFTP

Funcion FTP para cargar desde sitio FTP hacia un container 'cubit' en Storage Azure

In [2]:
# FTP Desde cubit
def ftp2blob():
    connFTP = init_FTP()

    ftp = FTP()
    ftp.connect(Host, Puerto, timeout=1000)
    ftp.login(user=Cuenta, passwd=Pass)

    files = ['GESTION.csv' ]
    logging.info('Se inicia descarga desde FTP')

    for j in range(len(files)):
        #filename = files[j]
        filename = "midata.csv"
        try:
            local_filename = os.path.join(r"C:/Users/paranedag/Downloads/temporal", filename)
            lf = open(local_filename, "wb")
            ftp.retrbinary("RETR " + filename, lf.write)
            lf.close()

            blob = BlobClient.from_connection_string(conn_str=connect_str, container_name='almacen', blob_name=filename)
            with open(file=local_filename, mode="rb") as file:
                blob.upload_blob(file, overwrite=True)

            logging.info("LISTA DESCARGA "+filename)
        except ValueError:
            logging.info("Error " + ValueError)
    logging.info('Término de descarga desde FTP')

In [6]:
# 
def azure_upload_df(container=None, dataframe=None, filename=None):
    if all([container, len(dataframe), filename]):
        file_path = f"{os.getenv('OUTPUT')}"
        upload_file_path = os.path.join(file_path, f"{filename}.csv")
        #connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        blob_client = blob_service_client.get_blob_client(
            container=container, blob=upload_file_path
        )
        try:
            output = dataframe.to_csv(index=False, encoding="utf-8", sep=";")
        except Exception as e:
            pass
        try:
            blob_client.upload_blob(output, blob_type="BlockBlob")
        except Exception as e:
            pass

def azure_download_csv_to_df(url=None):
    if url:
        #connect_str = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
        blob_service_client = BlobServiceClient.from_connection_string(connect_str)
        path = urlparse(url).path
        path = path.split("/")
        container = path[1]
        blob = '/'.join(path[2:])
        blob_client = blob_service_client.get_blob_client(container=container, blob=blob)
        with BytesIO() as input_blob:
            blob_client.download_blob().download_to_stream(input_blob)
            input_blob.seek(0)
            df = pd.read_csv(input_blob)
        return df
    else:
        return None

In [8]:
# dataframe

azure_upload_df(container='almacen', dataframe=df, filename='registros')

In [3]:
# ejecutar FTP
ftp2blob()

In [27]:
# carga de archivos dentro de Azure Storage
filename = "GESTION.csv"
mifile = os.path.join(r"C:/Users/paranedag/Downloads/almacen/", filename)

blob = BlobClient.from_connection_string(conn_str=connect_str, container_name="almacen", blob_name=filename)
exists = blob.exists()

#blob_service_client = BlobServiceClient.from_connection_string(connect_str)
#blob = blob_service_client.get_blob_client(container="cubit", blob=filename)

with open(mifile, "rb") as file:
    await blob.upload_blob(file, overwrite=True)



C:\Users\paranedag\AppData\Local\Temp\ipykernel_3592\434729430.py:6: RuntimeWarning: coroutine 'BlobClient.exists' was never awaited
  exists = blob.exists()
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x000001E0A53A7D30>
